In [1]:
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import os

# Crear una lista para rastrear URLs ya visitadas
visited_urls = set()

# Función recursiva para scrapear cada página
def scrape_page(url, folder_name="scraped_texts"):
    if url in visited_urls:
        return  # Evitar visitar la misma página varias veces
    
    # Añadir la URL a la lista de visitados
    visited_urls.add(url)
    
    # Realizar la solicitud a la página
    try:
        response = requests.get(url)
    except requests.RequestException as e:
        print(f"Error al cargar la página {url}: {e}")
        return

    # Verificar que la solicitud fue exitosa
    if response.status_code == 200:
        # Crear el objeto BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extraer y limpiar el texto
        text = soup.get_text()
        cleaned_text = re.sub(r'\s+', ' ', text).strip()
        
        # Guardar el texto limpio en un archivo .txt
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        file_name = f"{folder_name}/{url.replace('https://', '').replace('/', '_')}.txt"
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(cleaned_text)
        
        print(f"Texto exportado a {file_name}")
        
        # Encontrar todos los enlaces en la página
        links = [a['href'] for a in soup.find_all('a', href=True)]
        
        # Seguir cada enlace recursivamente
        for link in links:
            # Manejar enlaces relativos
            if link.startswith('/'):
                link = url + link
            elif not link.startswith('http'):
                continue  # Ignorar enlaces que no sean http o https
            scrape_page(link, folder_name)  # Llamada recursiva
    else:
        print(f"Error al cargar la página {url}: Status {response.status_code}")

# URL inicial
start_url = "https://www.medellin.travel/"
scrape_page(start_url)
